In [ ]:
# HWK 4 - "In-class IMDB Kaggle competition" 

# 1. Use the smaller sample data per above for your train/test split 
#    (note you may use the whole data set too!)
#
#    X_train = df.loc[:2500, 'review'].values
#    y_train = df.loc[:2500, 'sentiment'].values
#    X_test  = df.loc[2500:5000, 'review'].values
#    y_test  = df.loc[2500:5000, 'sentiment'].values

# 2. Employ any technique learnt in class or outside of class to arrive at 
#    the best ML engine model for predicting movie sentiment base on reviews
#
#    Do 5-fold cross-validation and report your best model's train/test accuracy, f1-score, and auc
#    The winner will be the ML algo model that has the highest f1-score on both the training and test data.
#    Just as in the regular Kaggle competition combining models will be acceptable (collusion is ok!). 
#    However, please turn in your notebook individually to Blackbaord. Let the games begin! 

In [66]:
import pyprind
import pandas as pd
import os

pbar = pyprind.ProgBar(50000)
labels = {'pos':1, 'neg':0}

df = pd.DataFrame()

for s in ('test','train'):
    for l in ('pos','neg'):
        path = '/Users/gavyn/Documents/BDML/Week 4/aclImdb/%s/%s' % (s,l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                try:
                    txt = infile.read()
                    df = df.append([ [txt, labels[l]] ], ignore_index=True)
                    pbar.update()
                except:
                    pass

df.columns = ['review', 'sentiment']

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:08:40


In [3]:
df

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1
5,"I saw this film on September 1st, 2005 in Indi...",1
6,"Maybe I'm reading into this too much, but I wo...",1
7,I felt this film did have many good qualities....,1
8,This movie is amazing because the fact that th...,1
9,"""Quitting"" may be as much about exiting a pre-...",1


In [67]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('/Users/gavyn/Documents/BDML/Week 4/aclImdb/movie_data.csv', index = False, encoding='utf-8')

In [68]:
import re
def preprocessor(text):
# find '<' then anything not '>' [^>], [^>]* 0 or more prefix, then close with '>'    
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text) 
    # eyes[:,;,=], optional nose [-], and mouth[),(,D,P)]
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [69]:
df['review'] = df['review'].apply(preprocessor)

In [70]:
def tokenizer(text):
    return text.split()

In [71]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [73]:
df = df.reset_index(drop=True)

In [79]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gavyn\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [22]:
# set df_stemmer to be the df with PorterStemmer applied

#df['review'] = df['review'].apply(tokenizer_porter)

#df = df.reset_index(drop=True)

In [75]:
# grab stuff

X_train = df.loc[:2500, 'review'].values
y_train = df.loc[:2500, 'sentiment'].values

X_test = df.loc[2500:5000, 'review'].values
y_test = df.loc[2500:5000, 'sentiment'].values

In [76]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

2501
2501
2501
2501


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None,
                       lowercase = False)

param_grid = [
              {'vect__ngram_range':[(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C':[1.0, 10.0, 100.0]}
                ]
lr_tfidf = Pipeline([ ('vect', tfidf) ,
                      ('clf',  LogisticRegression(random_state=0))])
gs_lr_tfidf = GridSearchCV( lr_tfidf, param_grid,
                          scoring = 'accuracy',
                          cv = 5, verbose = 1,
                          n_jobs = 1) # n_jobs -1 uses all computer cores
gs_lr_tfidf.fit(X_train, y_train)
print('The Best parameter set: %s' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f'
     % gs_lr_tfidf.best_score_)
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
###############################################

In [2]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def tokenizer(text):  # converts reviews to lower case, take out non-words, and put emoticons at the end
    text = re.sub('<[^>]*>', '', text) #
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8')  as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2]) 
            yield text, label 
            
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error = 'ignore',
                         n_features = 2**21,
                         preprocessor = None,
                         tokenizer = tokenizer)

clf = SGDClassifier(loss='log', random_state = 1, n_iter = 1)

#doc_stream = stream_docs(path = '/Users/jim/Desktop/aclImdb/movie_data.csv')
loc_file = '/Users/Dawn/Documents/Python Scripts/aclImdb/movie_data.csv'
doc_stream = stream_docs(path = loc_file)

  
#import pyprind
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes = classes) 
    
X_test, y_test = get_minibatch(doc_stream, size = 1000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))


Accuracy: 0.878


In [3]:

import pickle
import os
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(clf,
           open(os.path.join(dest, 'classifier1.pkl'),'wb'),
           protocol = 4)

pickle.dump(stop,
           open(os.path.join(dest, 'stopwords.pkl'),'wb'),
           protocol = 4)

In [ ]:
import pickle
import os
des=os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest):
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'),'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest,'classifier.pkl'), 'wb'), protocol=4)

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir=os.path.dirname(__file__)
stop=pickle.load(open(os.path.join(cur_dir,'pkl_objects','stopwords.pkl'),'rb'))

def tokenizer(text):
    text=re.sub('<[^>]*>','',text)
    emoticons=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text.lower())
    text=re.sub('[\W]+',' ',text.lower()) \
                +' '.join(emoticons).replace('-','')
    tokenized=[w for w in text.split() if w not in stop]
    return tokenized

vect=HashingVectorizer(decode_error='ignore',n_features=2**21,preprocessor=None, tokenizer=tokenizer)

In [ ]:
import pickle
import re
import os
from vectorizer import vect
clf=pickle.load(open(os.path.join('pkl_objects','classifier.pkl'),'rb'))

In [ ]:
import numpy as np
label={0:'negative',1:'positive'}
example=['I love this movie']
X=vect.transform(example)
print('Pridiction: %s\nProbability: %.2f%%' %\(label[clf.predict(X)[0]],np.max(clf.predict_proba(X))*100))

In [ ]:
conn=sqlite3.connect('reviews.sqlite')
c=conn.cursor()
c.execute("SELECT * FROM review_db WHERE date"\
         "BETWEEN '2015-01-01 00:00:00 AND DATETIME('now')")
results=c.fetchall()
conn.close()
print(results)

In [ ]:
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np


1st_flask_app_2/
    app.py
    static/
        style.css
    templates/
        _formhelpers.html
        first_app.html
        hello.html

from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators

app=Flask(__name__)

class HelloForm(Form):
    sayhello=TextAreaField('',[validators.DataRequired()])

@app.route('/')
def index():
    form=HelloForm(request.form)
    return render_template('first_app.html', form=form)

@app.route('/hello', methods=['POST'])
def hello():
    form=HelloForm(request.form)
    if request.method=='POST' and form.validate():
        name=request.form['sayhello']
        return render_template('hello.html', name=name)
    return render_template('first_app.html', form=form)

if __name__=='__main__':
    app.run(debug=True)

if __name__=='__main__':
    app.run()

In [ ]:
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np
from vectorizer import vect

app=Flask(__name__)

cur_dir=os.path.dirname(__file__)
clf=pickle.load(open(os.path.join(cur_dir,'pkl_objects/classifier.pkl'),'rb'))
db=os.path.join(cur_dir,'reviews.sqlite')

def classify(document):
    label={0:'negative',1:'positive'}
    X=vect.transform([document])
    y=clf.predict(X)[0]
    proba=np.max(clf.predict_proba(X))
    return label[y],proba

def train(document, y):
    X=vect.transform([document])
    clf.partial_fit(X,[y])
    
def sqlite_entry(path,document,y):
    conn=sqlite3.connect(path)
    c=conn.cursor()
    c.execute("INSERT INTO review_db(review,sentiment,date)"\"VALUES(?,?,DATETIME('now'))",(document,y))
    conn.commit()
    conn.close()
    
app=Flask(__name__)
class ReviewForm(Form):
    moviereview=TextAreaField('',[validators.DataRequired(),validators.length(min=15)])

@app.route('/')
def index():
    form=ReviewForm(request.form)
    return render_template('reviewform.html', form=form)

@app.route('/results', methods=['POST'])
def results():
    form=ReviewForm(request.form)
    if request.method=='POST' and form.validate():
        review=request.form['moviereview']
        y,proba=classify(review)
        return render_template('results.html', content=review,prediction=y,probability=round(proba*100,2))
    return render_template('reviewform.html', form=form)

@app.route('/thanks', methods=['POST'])
def feedback():
    feedback=request.form['feedback_button']
    review=request.form['review']
    prediction=request.form['prediction']
    inv_label={'negative':0,'positive':1}
    y=inv_label[prediction]
    if feedback=='Incorrect'
        y=int(not(y))
    train(review,y)
    sqlite_entry(db,review,y)
    return render_template('thanks.html')

if __name__=='__main__':
    app.run(debug=True)

In [ ]:
<!doctype html>
<html>
<head>
<title>Movie Classification</title>
</head>
<body>
<h2>Please enter your movie review:</h2>
{% from "_formhelpers.html" import render_field %}
<form method=post action="/results">
<dl>
{{ render_field(form.moviereview, cols='30', rows='10') }}
</dl>
<div>
<input type=submit value='Submit review' name='submit_btn'>
</div>
</form>
</body>
</html>
<!doctype html>
<html>
<head>
<title>Movie Classification</title>
<link rel="stylesheet" href="{{ url_for('static',
filename='style.css') }}">
</head>
<body>
<h3>Your movie review:</h3>
<div>{{ content }}</div>
<h3>Prediction:</h3>
<div>This movie review is <strong>{{ prediction }}</strong>
(probability: {{ probability }}%).</div>
<div id='button'>
<form action="/thanks" method="post">
<input type=submit value='Correct' name='feedback_button'>
<input type=submit value='Incorrect' name='feedback_button'>
<input type=hidden value='{{ prediction }}' name='prediction'>
<input type=hidden value='{{ content }}' name='review'>
</form>
</div>
<div id='button'>
<form action="/">
<input type=submit value='Submit another review'>
</form>
</div>
</body>
</html>

body{
width:600px;
}
#button{
padding-top: 20px;
}

<!doctype html>
<html>
<head>
<title>Movie Classification</title>
</head>
<body>
<h3>Thank you for your feedback!</h3>
<div id='button'>
<form action="/">
<input type=submit value='Submit another review'>
</form>
</div>
</body>
</html>

In [ ]:
def update_model(db_path,model,batch_size=10000):
    conn=sqlite3.connect(db_path)
    c=conn.cursor()
    c.execute('SELECT*from review_db')
    
    results=c.fetchmany(batch_size)
    while results:
        data=np.array(results)
        X=data[:,0]
        y=data[:,1].astype(int)
        classes=np.array([0,1])
        X_train=vect.transform(X)
        clf.partial_fit(X_train,y,classes=classes)
        results=c.fetchmany(batch_size)
        
    conn.close()
    return None

cur_dir=os.path.dirname(__file__)
clf=pickle.load(open(os.path.join(cur_dir,'pkl_objects','classifier.pkl'),'rb'))
db=os.path.join(cur_dir,'reviews.sqlite')
update_model(db_path=db,model=clf,batch_size=10000)